$f(x,y) = x^{2}y+y+2 $

生成computation graph

In [1]:
import warnings
warnings.filterwarnings("ignore")
import tensorflow as tf

x = tf.Variable(3, name="x")
y = tf.Variable(4, name="y")
f = x*x*y + y + 2

打开TensorFlow session,并初始化变量x,y和计算f

In [2]:
#方法1
sess = tf.Session()
sess.run(x.initializer)
sess.run(y.initializer)
result = sess.run(f)
print(result)

42


In [3]:
#方法2  
with tf.Session() as sess:
    x.initializer.run()
    y.initializer.run()
    result = f.eval()
print(result)

42


除了手动初始化每个变量,还可以使用global_variables_initializer()  
该方法并不立即执行初始化,只是创建了图中的一个节点  
在run的时候,所有变量将会初始化

In [4]:
init = tf.global_variables_initializer()  # prepare an init node

with tf.Session() as sess:
    init.run()                 # actually initialize all the variables
    result = f.eval()
print (result)

42


### Node Value(节点值)的生命周期

In [5]:
w = tf.constant(3)
x = w + 2
y = x + 5
z = x * 3

with tf.Session() as sess:
    print(y.eval())  # 10
    print(z.eval())  # 15

10
15


在每次graph运行后,所有的节点值都会被丢弃,而变量值则不会丢弃  
因此在评估y和z的时候, w和x都被计算了2次

如果希望得到更有效率的计算,必须要求TensorFlow在一个graph中评估y和z

In [6]:
with tf.Session() as sess:
    y_val, z_val = sess.run([y, z])
    print(y_val)  # 10
    print(z_val)  # 15

10
15


### 使用公式法计算线性回归

In [ ]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np
from sklearn.datasets import fetch_california_housing
import tensorflow as tf

housing = fetch_california_housing()
m, n = housing.data.shape
housing_data_plus_bias = np.c_[np.ones((m, 1)), housing.data]

X = tf.constant(housing_data_plus_bias, dtype=tf.float32, name="X")
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name="y")
XT = tf.transpose(X)
theta = tf.matmul(tf.matmul(tf.matrix_inverse(tf.matmul(XT, X)), XT), y)

with tf.Session() as sess:
    theta_value = theta.eval()

该方法相比直接使用numpy进行公式法计算的好处是:  
一旦安装了GPU支持的Tensorflow,可以自动在GPU上进行运行

### 使用TensorFlow实现梯度下降

In [ ]:
n_epochs = 1000
learning_rate = 0.01

X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name="X")
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name="y")
theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0, seed=42), name="theta")
y_pred = tf.matmul(X, theta, name="predictions")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name="mse")
gradients = 2/m * tf.matmul(tf.transpose(X), error)
training_op = tf.assign(theta, theta - learning_rate * gradients)

init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)

    for epoch in range(n_epochs):
        if epoch % 100 == 0:
            print("Epoch", epoch, "MSE =", mse.eval())
        sess.run(training_op)
    
    best_theta = theta.eval()

random_uniform()生成一个包含随机值的张量,类似numpy中的rand()  
tf.random_uniform([n + 1, 1], -1.0, 1.0, seed=42) :  
表示生成(n+1)×1的矩阵,范围从-1到1,随机种子为42  
  
assign()函数创建了一个node,将对变量赋予一个新值

### 使用autodiff

按照上面的方法,需要输入推到的公式:  
gradients = 2/m * tf.matmul(tf.transpose(X), error)  
但在实际中,非常不实用,

In [ ]:
n_epochs = 1000
learning_rate = 0.01

X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name="X")
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name="y")
theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0, seed=42), name="theta")
y_pred = tf.matmul(X, theta, name="predictions")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name="mse")

In [ ]:
gradients = tf.gradients(mse, [theta])[0]   

使用tf.gradients自动计算梯度, 应变量为mse变量, 自变量为theta  
自变量可以为多个变量的列表形式

In [ ]:
training_op = tf.assign(theta, theta - learning_rate * gradients)

init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)

    for epoch in range(n_epochs):
        if epoch % 100 == 0:
            print("Epoch", epoch, "MSE =", mse.eval())
        sess.run(training_op)
    
    best_theta = theta.eval()

print("Best theta:")
print(best_theta)

### 使用Optimizer

TensorFlow提供了一种更简单的方法

In [ ]:
n_epochs = 1000
learning_rate = 0.01

X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name="X")
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name="y")
theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0, seed=42), name="theta")
y_pred = tf.matmul(X, theta, name="predictions")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name="mse")

In [ ]:
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(mse)

In [ ]:
init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)

    for epoch in range(n_epochs):
        if epoch % 100 == 0:
            print("Epoch", epoch, "MSE =", mse.eval())
        sess.run(training_op)
    
    best_theta = theta.eval()

print("Best theta:")
print(best_theta)

除了梯度下降法,还可以使用momentum optimizer  
仅需修改optimizer一行

In [ ]:
optimizer = tf.train.MomentumOptimizer(learning_rate=learning_rate,
                                       momentum=0.9)